In [101]:
import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_absolute_percentage_error

In [102]:
import joblib

In [103]:
model = joblib.load("final_model.pkl")

In [104]:
company = yf.Ticker('MSFT')

In [105]:
df = company.history(period='1mo',interval='15m')

In [106]:
t = []
for i in range(len(df)):
    if(df['Open'].iloc[i]-df['Close'].iloc[i]>0):
        t.append("DOWN")
    else:
        t.append("UP")
df['Trend']=t

In [107]:
th=[0]
current_trend = df['Trend'].iloc[0]
count = 0
for i in range(1,len(df)):
    if(current_trend==df['Trend'].iloc[i]):
        count+=1
        th.append(count)
    else:
        count = 0
        th.append(count)
        current_trend = df['Trend'].iloc[i]
        
df["Trend history"]=th

In [108]:
p=[]
v=[]
if(df['Trend'].iloc[0]=="DOWN"):
    peak = df['Open'].iloc[0]
    valley = df['Close'].iloc[0]
else:
    valley = df['Open'].iloc[0]
    peak = df['Close'].iloc[0]

for i in range(0,len(df)):
    
    p.append(peak-df['Close'].iloc[i])
    v.append(valley-df['Close'].iloc[i])
    
    if peak<df['Close'].iloc[i]:
        peak = df['Close'].iloc[i]
    if valley>df['Close'].iloc[i]:
        valley = df['Close'].iloc[i]
df['Change since last peak']=p
df['Change since last drop']=v        

In [109]:
c=[]
cm=[]
for i in range(0,len(df)):
    
    c.append(df['Open'].iloc[i]-df['Close'].iloc[i])
    cm.append(df['High'].iloc[i]-df['Low'].iloc[i])
    
df['Local Change']=c
df['Local range of stock price']=cm

In [110]:
df = pd.get_dummies(df,drop_first=True)

In [111]:
next_day_change = df['Local Change'][1:]

In [112]:
df = df[1:]
df["Next day prediction of change"]=next_day_change

In [113]:
X = df.drop(['Next day prediction of change'],axis=1)
y = df["Next day prediction of change"]

In [114]:
polynomial_converter = PolynomialFeatures(degree=2,include_bias=False)
poly_features = polynomial_converter.fit_transform(X)

pred = model.predict(poly_features)

test_RMSE = mean_absolute_percentage_error(y,pred)

In [115]:
test_RMSE

107288703.14868754